Criando um dataset códificado

In [10]:
import pandas as pd
import pandas as pd

# Carregar o dataset padronizado
df_scaled = pd.read_parquet('../data/scaled_ACC_INTAKES_OUTCOMES.parquet')

# Selecionar as colunas especificadas com os espaços nos nomes
selected_columns = [
    "age_upon_outcome_(years)", "outcome_month", "outcome_year", "dob_year", "dob_month", 
    "age_upon_intake_(years)", "intake_month", "intake_year", "time_in_shelter_days",
    "outcome_subtype_Aggressiv", "outcome_subtype_At Vet   ", "outcome_subtype_Barn     ", 
    "outcome_subtype_Behavior ", "outcome_subtype_Court/Inv", "outcome_subtype_Enroute  ", 
    "outcome_subtype_Foster   ", "outcome_subtype_In Foster", "outcome_subtype_In Kennel", 
    "outcome_subtype_In Surger", "outcome_subtype_Medical  ", "outcome_subtype_Offsite  ", 
    "outcome_subtype_Partner  ", "outcome_subtype_Possible ", "outcome_subtype_Rabies Ri", 
    "outcome_subtype_SCRP     ", "outcome_subtype_Snr      ", "outcome_subtype_Suffering", 
    "outcome_subtype_Underage ", "outcome_type_Adoption       ", "outcome_type_Died           ", 
    "outcome_type_Euthanasia     ", "outcome_type_Missing        ", "outcome_type_Return to Owner", 
    "outcome_type_Transfer       ", "sex_upon_outcome_Intact Female", "sex_upon_outcome_Intact Male  ", 
    "sex_upon_outcome_Neutered Male", "sex_upon_outcome_Spayed Female", "sex_upon_outcome_Unknown      ", 
    "outcome_weekday_Friday   ", "outcome_weekday_Monday   ", "outcome_weekday_Saturday ", 
    "outcome_weekday_Sunday   ", "outcome_weekday_Thursday ", "outcome_weekday_Tuesday  ", 
    "outcome_weekday_Wednesday", "animal_type_Bir", "animal_type_Cat", "animal_type_Dog", 
    "animal_type_Oth", "intake_condition_Aged   ", "intake_condition_Feral  ", 
    "intake_condition_Injured", "intake_condition_Normal ", "intake_condition_Nursing", 
    "intake_condition_Other  ", "intake_condition_Pregnan", "intake_condition_Sick   ", 
    "intake_type_Euthanasia Request", "intake_type_Owner Surrender   ", "intake_type_Public Assist     ", 
    "intake_type_Stray             ", "intake_type_Wildlife          ", "sex_upon_intake_Intact Female", 
    "sex_upon_intake_Intact Male  ", "sex_upon_intake_Neutered Male", "sex_upon_intake_Spayed Female", 
    "sex_upon_intake_Unknown      ", "intake_weekday_Friday   ", "intake_weekday_Monday   ", 
    "intake_weekday_Saturday ", "intake_weekday_Sunday   ", "intake_weekday_Thursday ", 
    "intake_weekday_Tuesday  ", "intake_weekday_Wednesday", "age_upon_outcome_age_group_encoded", 
    "age_upon_intake_age_group_encoded", "is_mix_breed", "color_Black", "color_Brown/Chocolate", 
    "color_Gray/Blue", "color_Other_Colors", "color_Patterned", "color_Red/Orange", 
    "color_White", "color_Yellow/Gold/Cream"
]

# Criar o novo dataset apenas com as colunas selecionadas
df_selected = df_scaled[selected_columns]

# Salvar o novo dataset codificado em formato Parquet
df_selected.to_parquet('../data/codificado_ACC_INTAKES_OUTCOMES.parquet')

# Verificar as primeiras linhas do novo dataset
df_selected.head()



,age_upon_outcome_(years),outcome_month,outcome_year,dob_year,dob_month,age_upon_intake_(years),intake_month,intake_year,time_in_shelter_days,outcome_subtype_Aggressiv,...,age_upon_intake_age_group_encoded,is_mix_breed,color_Black,color_Brown/Chocolate,color_Gray/Blue,color_Other_Colors,color_Patterned,color_Red/Orange,color_White,color_Yellow/Gold/Cream
0,10.0,4,2014,2004,4,10.0,4,2014,5.0,False,...,8,0,False,False,False,False,True,False,False,False
1,14.0,12,2014,2000,1,14.0,12,2014,14.1,False,...,2,1,False,False,False,True,False,False,False,False
2,16.0,11,2013,1997,10,16.0,10,2013,49.7,False,...,3,1,False,False,False,True,False,False,False,False
3,14.0,2,2015,2000,4,14.0,2,2015,0.1,False,...,2,1,False,False,False,True,False,False,False,False
4,17.0,10,2016,1999,6,17.0,9,2016,9.0,False,...,3,0,False,False,False,True,False,False,False,False


In [31]:
# Manter apenas as colunas das classes de interesse
df_filtered = df_selected[[
    "outcome_type_Adoption       ",
    "outcome_type_Euthanasia     ",
    "outcome_type_Transfer       "
]]

# Verificar a distribuição das classes
distribution = {
    "Adoption": df_filtered["outcome_type_Adoption       "].sum(),
    "Euthanasia": df_filtered["outcome_type_Euthanasia     "].sum(),
    "Transfer": df_filtered["outcome_type_Transfer       "].sum()
}

# Calcular a proporção de cada classe em relação ao total
total_instances = len(df_filtered)
proportions = {key: (value / total_instances) * 100 for key, value in distribution.items()}

# Exibir os resultados
print("Distribuição das Classes de Saída:")
for key, value in distribution.items():
    print(f"{key}: {value} instâncias ({proportions[key]:.2f}%)")


Distribuição das Classes de Saída:
Adoption: 5806 instâncias (16.90%)
Euthanasia: 5907 instâncias (17.19%)
Transfer: 22044 instâncias (64.16%)


In [32]:
from sklearn.model_selection import train_test_split

# Separar as features (X) e a variável alvo (y)
X = df_selected.drop(columns=[
    "outcome_type_Adoption       ",
    "outcome_type_Euthanasia     ",
    "outcome_type_Transfer       "
])
y = df_selected[[
    "outcome_type_Adoption       ",
    "outcome_type_Euthanasia     ",
    "outcome_type_Transfer       "
]]

# Dividir o dataset em treino e teste, mantendo a proporção das classes (estratificado)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Verificar as dimensões dos conjuntos de treino e teste
print("Dimensões do conjunto de treino:", X_train.shape, y_train.shape)
print("Dimensões do conjunto de teste:", X_test.shape, y_test.shape)


Dimensões do conjunto de treino: (27486, 83) (27486, 3)
Dimensões do conjunto de teste: (6872, 83) (6872, 3)


Vamos proceder com o balanceamento do conjunto de treino usando uma combinação de oversampling (para aumentar as classes minoritárias) e undersampling (para reduzir a classe majoritária).

In [34]:
import numpy as np
from imblearn.combine import SMOTEENN

# Converter o DataFrame y_train em um array 1D com os rótulos das classes
y_train_labels = y_train.values.argmax(axis=1)

# Combinação de SMOTE para oversampling e Edited Nearest Neighbours para undersampling
smote_enn = SMOTEENN(sampling_strategy='auto', random_state=42)

# Aplicar o SMOTEENN para balancear o conjunto de treino
X_resampled, y_resampled = smote_enn.fit_resample(X_train, y_train_labels)

# Verificar a nova distribuição das classes no conjunto de treino balanceado
print("Distribuição das classes após balanceamento:")
print(np.bincount(y_resampled))



Distribuição das classes após balanceamento:
[16944 16979 13788]


In [35]:
# Mapear os valores para as classes
classes = ["Adoption", "Euthanasia", "Transfer"]
class_distribution = dict(zip(classes, np.bincount(y_resampled)))

# Exibir a distribuição das classes após o balanceamento
print("Distribuição das classes após balanceamento:")
for class_name, count in class_distribution.items():
    print(f"{class_name}: {count} instâncias")


Distribuição das classes após balanceamento:
Adoption: 16944 instâncias
Euthanasia: 16979 instâncias
Transfer: 13788 instâncias


In [37]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Instanciar o modelo Random Forest
rf_model = RandomForestClassifier(random_state=42)

# Treinar o modelo com o conjunto de treino balanceado
rf_model.fit(X_resampled, y_resampled)

# Fazer previsões no conjunto de teste
y_pred = rf_model.predict(X_test)

# Converter y_test para um array 1D com os rótulos das classes
y_test_labels = y_test.values.argmax(axis=1)

# Avaliar o modelo
accuracy = accuracy_score(y_test_labels, y_pred)
print(f"Acurácia: {accuracy}")

# Relatório de classificação
print("\nRelatório de Classificação:")
print(classification_report(y_test_labels, y_pred, target_names=["Adoption", "Euthanasia", "Transfer"]))

# Matriz de Confusão
print("Matriz de Confusão:")
print(confusion_matrix(y_test_labels, y_pred))


Acurácia: 0.9991268917345751

Relatório de Classificação:
              precision    recall  f1-score   support

    Adoption       1.00      1.00      1.00      1281
  Euthanasia       1.00      1.00      1.00      1182
    Transfer       1.00      1.00      1.00      4409

    accuracy                           1.00      6872
   macro avg       1.00      1.00      1.00      6872
weighted avg       1.00      1.00      1.00      6872

Matriz de Confusão:
[[1278    2    1]
 [   2 1179    1]
 [   0    0 4409]]


In [39]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Instanciar o modelo XGBoost
xgb_model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')

# Treinar o modelo com o conjunto de treino balanceado
xgb_model.fit(X_resampled, y_resampled)

# Fazer previsões no conjunto de teste
y_pred = xgb_model.predict(X_test)

# Avaliar o modelo
accuracy = accuracy_score(y_test_labels, y_pred)
print(f"Acurácia: {accuracy}")

# Relatório de classificação
print("\nRelatório de Classificação:")
print(classification_report(y_test_labels, y_pred, target_names=["Adoption", "Euthanasia", "Transfer"]))

# Matriz de Confusão
print("Matriz de Confusão:")
print(confusion_matrix(y_test_labels, y_pred))


c:\Users\julia\Documents\UFRPE\PROJETOS3\projetos3_novo_julia\projetos3-julia\env\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:19:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Acurácia: 0.9998544819557625

Relatório de Classificação:
              precision    recall  f1-score   support

    Adoption       1.00      1.00      1.00      1281
  Euthanasia       1.00      1.00      1.00      1182
    Transfer       1.00      1.00      1.00      4409

    accuracy                           1.00      6872
   macro avg       1.00      1.00      1.00      6872
weighted avg       1.00      1.00      1.00      6872

Matriz de Confusão:
[[1280    1    0]
 [   0 1182    0]
 [   0    0 4409]]


In [40]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Instanciar o modelo SVM
svm_model = SVC(kernel='linear', random_state=42)

# Treinar o modelo com o conjunto de treino balanceado
svm_model.fit(X_resampled, y_resampled)

# Fazer previsões no conjunto de teste
y_pred = svm_model.predict(X_test)

# Avaliar o modelo
accuracy = accuracy_score(y_test_labels, y_pred)
print(f"Acurácia: {accuracy}")

# Relatório de classificação
print("\nRelatório de Classificação:")
print(classification_report(y_test_labels, y_pred, target_names=["Adoption", "Euthanasia", "Transfer"]))

# Matriz de Confusão
print("Matriz de Confusão:")
print(confusion_matrix(y_test_labels, y_pred))


Acurácia: 0.9960710128055879

Relatório de Classificação:
              precision    recall  f1-score   support

    Adoption       0.99      1.00      0.99      1281
  Euthanasia       1.00      0.99      1.00      1182
    Transfer       1.00      1.00      1.00      4409

    accuracy                           1.00      6872
   macro avg       0.99      1.00      0.99      6872
weighted avg       1.00      1.00      1.00      6872

Matriz de Confusão:
[[1276    3    2]
 [   3 1174    5]
 [  14    0 4395]]


In [42]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

# Definir os modelos
models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss'),
    "SVM": SVC(kernel='linear', random_state=42)
}

# Armazenar os resultados
results = {}

# Realizar a validação cruzada para cada modelo
for model_name, model in models.items():
    cv_scores = cross_val_score(model, X_resampled, y_resampled, cv=5)
    results[model_name] = {
        "mean_accuracy": cv_scores.mean(),
        "std_accuracy": cv_scores.std(),
        "cv_scores": cv_scores
    }

# Exibir os resultados
for model_name, result in results.items():
    print(f"Modelo: {model_name}")
    print(f"Acurácias em cada fold: {result['cv_scores']}")
    print(f"Média da Acurácia: {result['mean_accuracy']}")
    print(f"Desvio Padrão da Acurácia: {result['std_accuracy']}\n")


c:\Users\julia\Documents\UFRPE\PROJETOS3\projetos3_novo_julia\projetos3-julia\env\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:34:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\julia\Documents\UFRPE\PROJETOS3\projetos3_novo_julia\projetos3-julia\env\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:34:36] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\julia\Documents\UFRPE\PROJETOS3\projetos3_novo_julia\projetos3-julia\env\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:34:37] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboos

Modelo: Random Forest
Acurácias em cada fold: [0.99979042 0.9997904  1.         1.         1.        ]
Média da Acurácia: 0.9999161645268853
Desvio Padrão da Acurácia: 0.00010267706597275027

Modelo: XGBoost
Acurácias em cada fold: [0.99958084 0.9998952  1.         1.         1.        ]
Média da Acurácia: 0.999895208953163
Desvio Padrão da Acurácia: 0.00016233818914911642

Modelo: SVM
Acurácias em cada fold: [0.99748507 0.9975896  0.9998952  0.9997904  0.9997904 ]
Média da Acurácia: 0.9989101344567682
Desvio Padrão da Acurácia: 0.0011220255908744184

